In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#del df
df = pd.read_csv("/kaggle/input/ip-network-traffic-flows-labeled-with-87-apps/Dataset-Unicauca-Version2-87Atts.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df['ProtocolName'].value_counts()
#so total 78 protocol names avialable 

In [ ]:
#checking 20-80% rule here 
np.cumsum((df['ProtocolName'].value_counts()/df.shape[0]).head(15))
#from the data we can say it is appliacable. Here 10 apps contribute to 94.3%, for safer side taking 15 apps which is 97.8%

In [ ]:
#by taking this data itself will describe the whole dataset.
print(np.cumsum((df['ProtocolName'].value_counts()/df.shape[0]).tail(62)))
#other applications are considered as low or rarely used. Since the cumulative sum is less then 2%
prtnthelping = list(df['ProtocolName'].value_counts().tail(78-16).index)

In [ ]:
#dropping the entries with this applications 
print(df[df['ProtocolName'].isin(prtnthelping)].shape, df.shape)

In [ ]:
df.describe()

In [ ]:
df['Source.IP'].value_counts()

In [ ]:
df[df['Source.IP'].str.contains('10.200.7.') | (df['Source.IP'].str.contains('192.168.')) ].shape[0]/df.shape[0]

In [ ]:
#classifing the users baased on flow rates
#df[df['Flow.Bytes.s'] > (1024*8*2048)]['Source.IP'].value_counts()
# 916 high profiles
#df[(df['Flow.Bytes.s'] < (1024*8*2048)) & (df['Flow.Bytes.s'] > (1024*8*1024))]['Source.IP'].value_counts()
# 967 medium profiles
#df[(df['Flow.Bytes.s'] < (1024*8*1024))]['Source.IP'].value_counts()
# 3570 low profile
#need to remove the duplicate entries
hiprof =  list(df[df['Flow.Bytes.s'] > (1024*8*2048)]['Source.IP'].value_counts().index)
med1prof = list(df[(df['Flow.Bytes.s'] < (1024*8*2048)) & (df['Flow.Bytes.s'] > (1024*8*1024))]['Source.IP'].value_counts().index)
med2prod =  list(df[(df['Flow.Bytes.s'] < (1024*8*1024)) & (df['Flow.Bytes.s'] > (1024*8*256))]['Source.IP'].value_counts().index)
lowprof = list(df[(df['Flow.Bytes.s'] < (1024*8*256))]['Source.IP'].value_counts().index)

In [ ]:
def labels(x):
    if(hiprof.count(x)):
        return 3
    elif(med1prof.count(x)):
        return 2
    elif(med2prod.count(x)):
        return 1
    else:
        return 0
df['labels'] = df['Source.IP'].apply(lambda x: labels(x))

In [ ]:
df[df['Source.IP'].str.contains('192.168.')]['Source.IP'].value_counts()[df[df['Source.IP'].str.contains('192.168.')]['Source.IP'].value_counts() > 3000]

In [ ]:
#the ip can be majorly classified by two major Ip subnets
#'10.200.7.0' mask 255.255.255.0
#'192.168.0.0' mask 255.255.0.0
# around 91% traffic collected from this traffic 

In [ ]:
np.cumsum((df[df['Source.IP'].str.contains('10.200.7.') | (df['Source.IP'].str.contains('192.168.'))]['ProtocolName'].value_counts().tail(62))/df.shape[0])

In [ ]:
prtin = df[df['Source.IP'].str.contains('10.200.7.') | (df['Source.IP'].str.contains('192.168.'))]['ProtocolName'].value_counts().head(16).index

In [ ]:
df[~df['ProtocolName'].isin(prtin)]

In [ ]:
# QoS is categorised based on source/destination port or IP
# flags will not help us in categorsing these dateset. 
# in the same way, recieved/Bwd will not help us here. For betterment of understanding we can avoid it(controlling the receiving end will reduce the quality of stream or browsing)
# active and idle state can be avoided
# header length of the packet will not help us, because it remains almost same to all entries
# so I am taking the for particlular user(source IP) what will be average packet size received sent or received to be known. along with it 
#      flow duartion, bytes/sec, protocolname are used. and other columns are dropped
df.columns

In [ ]:
df[['Source.IP','Average.Packet.Size','Flow.Duration','Flow.Bytes.s','ProtocolName','labels']].head()

In [ ]:
#I would like to categorise the dataset into 3. 
# 1. less data/commonly consuming apps
# 2. commomnly used apps
# 3. high priority needed apps
# why we need the above, because the dataset follows 20-90% ratio concept. There are around 62 apps that are less consumed or needed less visibility
# There are 3-5 apps commonly used, this speed should not be disturbed
# some apps need high data to run, this needs to be consentrated. 

In [ ]:
#EDA to be done in categorising apps

In [ ]:
df = df[['Source.IP','Average.Packet.Size','Flow.Duration','Flow.Bytes.s','ProtocolName','labels']]
df1 = df[~df['ProtocolName'].isin(prtnthelping)]
df1.head()

In [ ]:
print("apps: ",df['ProtocolName'].value_counts().index)

In [ ]:
protocolbased = ['SOCKS','MAIL_IMAPS', 'SSL', 'SSL_NO_CERT','RADIUS', 'OPENVPN', 'SNMP', 'IP_OSPF','BGP','FTP_CONTROL', 'NTP', 'SSH', 'FTP_DATA', 'DNS', 'IP_ICMP','SSL', 'NFS', 'RTMP']
#relabel http  ['HTTP','HTTP_PROXY', 'HTTP_CONNECT',  'HTTP_DOWNLOAD'] components unders browsing
#relabel APPLE components  APPLE_ITUNES, APPLE_ICLOUD, APPLE 
http = ['HTTP','HTTP_PROXY', 'HTTP_CONNECT',  'HTTP_DOWNLOAD']
def appsrelabel(x):
    if(http.count(x)):
        return 'Browsing'
    else:
        return x
df['ProtocolName'] = df['ProtocolName'].apply(lambda x: appsrelabel(x))
print("protocol based apps priority should not be set, this packets:" ,protocolbased)
print("removing the protocol apps from list",df[~df['ProtocolName'].isin(protocolbased)]['ProtocolName'].value_counts().index)

In [ ]:
df = df[~df['ProtocolName'].isin(protocolbased)]

In [ ]:
#now we have only the apps based entries in the list, checking again is passing 20-80

In [ ]:
np.cumsum((df['ProtocolName'].value_counts()/df.shape[0]).head(12))
#based on entries.

In [ ]:
df.groupby(by = 'ProtocolName')[['Flow.Bytes.s','Average.Packet.Size','Flow.Duration']].sum().sort_values(by=['Flow.Duration','Flow.Bytes.s'], ascending =False)

In [ ]:
df.groupby(by = 'ProtocolName')[['Flow.Bytes.s','Average.Packet.Size','Flow.Duration']].sum().sort_values(by=['Flow.Bytes.s'], ascending =False)

In [ ]:
df.groupby(by = 'ProtocolName')[['Flow.Bytes.s','Average.Packet.Size','Flow.Duration']].mean().sort_values(by=['Flow.Bytes.s','Flow.Duration'], ascending =False)

In [ ]:
#mapping the labels with apps
print(pd.crosstab(df['ProtocolName'],df['labels']))
df1 = pd.crosstab(df['ProtocolName'],df['labels'])
print("most commonly used apps are: ",'GOOGLE','Browsing', 'YOUTUBE', 'AMAZON')

In [ ]:
print('apps highly used by high consumers',df1[0].sort_values(ascending = False).head(14).index)
print('apps highly used by medium consumers',df1[1].sort_values(ascending = False).head(14).index)
print('apps highly used by medium consumers',df1[1].sort_values(ascending = False).head(6).index)
#using value 13 for 20-80 raito. Above which windows update is not a app, so adding +1 to 13

In [ ]:
df.head()

In [ ]:
sns.countplot(data = df, x= 'labels')
plt.show()
plt.figure(figsize=(10,12))
chart = sns.countplot(data = df, x='ProtocolName')
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
#classifing the users baased on flow duration
#need to remove the duplicate entries
df2 = pd.crosstab(columns=df['ProtocolName'],values= df['Flow.Duration'],index=df['Source.IP'],aggfunc='max',dropna=True)
df2 = df2.fillna(0).sum()

In [ ]:
df2.sort_values(ascending=False).head(15)

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score,confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score,KFold

In [ ]:
X = df[['Average.Packet.Size', 'Flow.Duration', 'Flow.Bytes.s']]
X = pd.DataFrame(StandardScaler().fit_transform(X),columns= ['Average.Packet.Size', 'Flow.Duration', 'Flow.Bytes.s'])
X['ProtocolName']= LabelEncoder().fit_transform(df['ProtocolName'])
X['Source.IP']= LabelEncoder().fit_transform(df['Source.IP'])

In [ ]:
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, init='random',n_init=10, random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)

plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
km = KMeans(n_clusters= 4, init='random',n_init=10, random_state=0)
km.fit(X)
df['klabel'] = km.labels_

In [ ]:
print(confusion_matrix(df['labels'],km.labels_))
print(classification_report(df['labels'],km.labels_))

In [ ]:
# The scores are very low because in the above steps, in manual labeling the high and medium are split into 3 and kmeans is 4. 
# we need to split into three and check the scores  again.